# TF-IDF of essays 
This jupyter notebook takes in a pool of essays, and constructs the TF-IDF score and trains a neural network to take in a TF-IDF matrix and compute a vocabulary score based on this information

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
from sklearn.feature_extraction.text import TfidfVectorizer
import math
import re
'''
this is the way to get kaggle files I think
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''

#hello
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"\nthis is the way to get kaggle files I think\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n"

In [4]:
#read in csv file
full_train_data = pd.read_csv('./data/train.csv', index_col=0)

In [5]:
#parse over every row, store essay in the essays_train/test lists, and store the 6 scores in the scores_train/test list
essays_train = []
scores_train = []
essays_test = []
scores_test = []
all_essays = []
all_vocabs_words = []

#Technique for splitting essays into training and testing data and results

for ind, data in enumerate(full_train_data.iterrows()):
    text, cohes, syntax, vocab, phrase, gram, convs = data[1]
    essays_train.append(text) if ind % 2 == 0 else essays_test.append(text)
    scores_train.append([cohes, syntax, vocab, phrase, gram, convs]) if ind % 2 == 0 else scores_test.append(
        [cohes, syntax, vocab, phrase, gram, convs]) 
'''
for ind, data in enumerate(full_train_data.iterrows()):
    text, cohes, syntax, vocab, phrase, gram, convs = data[1]
    all_essays.append(text)
    all_vocabs_words.append(vocab) '''


'\nfor ind, data in enumerate(full_train_data.iterrows()):\n    text, cohes, syntax, vocab, phrase, gram, convs = data[1]\n    all_essays.append(text)\n    all_vocabs_words.append(vocab) '

In [6]:
#make into np arrays, currently not used
np_essays_train = np.array(essays_train)
np_essays_test = np.array(essays_test)
np_scores_train = np.array(scores_train)
np_scores_test = np.array(scores_test)

In [7]:
#make tfIdf for all essays in bank
#np_all_essays = np.concatenate((essays_train, essays_test), axis = 0)
# not use in test approach
np_all_essays = np.array(all_essays)
np_all_scores = np.array(all_vocabs_words)

Test code new approach

In [9]:
#Reads in 333,000 most common words in English language, not currently used later in code but might be useful...
all_vocab_words = pd.read_csv('./data/unigram_freq.csv', index_col=0)
all_vocabs = np.array([data[0] for data in all_vocab_words.iterrows()])

In [17]:
# building dictionary with vocab word as key and count as value
vocab_dict = {}

for data in all_vocab_words[:50001].iterrows():
  vocab_dict[data[0]] = data[1][0]

In [18]:
vocab_dict["the"]

23135851162

In [19]:
normalizer_frequency = vocab_dict["the"]

In [20]:
vocab_word_to_index = {}
index_to_vocab_word = {}
for index, word in enumerate(vocab_dict):
  vocab_word_to_index[word] = index
  index_to_vocab_word[index] = word

In [68]:
our_idf = []
#create idf for every word:
for word, amount in vocab_dict.items():
  idf_score = math.log((normalizer_frequency + 1)/amount)
  our_idf.append(idf_score)

np_idf = np.array(our_idf)

In [22]:
unknown_words = []
train_tf_essays = []
for essay in np_essays_train:
  essay_wo_punc = re.sub(r'[^\w\s]', '', essay)
  essay_lower = essay_wo_punc.lower()
  split_essay = re.split('[^a-zA-Z]+', essay_lower)
  tf = [0 for _ in range(50000)]
  #calculate TF-IDF Score
  for word in split_essay:
    if word in vocab_dict:
      tf[vocab_word_to_index[word]] += 1
    else:
      unknown_words.append(word)
  train_tf_essays.append(tf)
np_train_tf_essays = np.array(train_tf_essays)

In [34]:
test_tf_essays = []
for essay in np_essays_test:
  essay_wo_punc = re.sub(r'[^\w\s]', '', essay)
  essay_lower = essay_wo_punc.lower()
  split_essay = re.split('[^a-zA-Z]+', essay_lower)
  tf = [0 for _ in range(50000)]
  #calculate TF-IDF Score
  for word in split_essay:
    if word in vocab_dict:
      tf[vocab_word_to_index[word]] += 1
    else:
      unknown_words.append(word)
  test_tf_essays.append(tf)
np_test_tf_essays = np.array(test_tf_essays)

In [23]:
#interesting data vizualization
unknown_words

['homebecause',
 'theyll',
 '',
 'aint',
 'kindess',
 'homeles',
 'triying',
 'greate',
 'kindess',
 'wount',
 'thoose',
 'kindess',
 'kindess',
 'trowing',
 'kindess',
 '',
 '',
 'isent',
 'isyou',
 'beatifull',
 'isent',
 'selfcondifence',
 'impotently',
 'selfcondience',
 'selfcondicence',
 'fild',
 'selfcondidence',
 'negitive',
 'incuage',
 'disagreedlooking',
 'highschol',
 '',
 'grocerys',
 'thenthey',
 'intead',
 'aruge',
 'techbology',
 'postive',
 'thngs',
 '',
 'coronavirus',
 'expensives',
 'activitys',
 'kahootquizlet',
 '',
 'costumer',
 'costumer',
 'costumer',
 '',
 'pedophiles',
 '',
 'interducaions',
 'fanfeeling',
 'enjoble',
 'consecrate',
 'anther',
 'paragargh',
 '',
 'einsteins',
 'einsteins',
 'weve',
 'einsteins',
 'hypothesize',
 'einsteins',
 'weve',
 'weve',
 '',
 'insted',
 'tradional',
 'learnd',
 'afther',
 'afther',
 'afther',
 'restaurent',
 'clases',
 'insted',
 'clases',
 'enought',
 'learnd',
 'studnet',
 'differnt',
 'yearsmay',
 'laern',
 'differen

In [69]:
np_idf[0] = .5

In [70]:
print(len(np_train_tf_essays[0]))
print(len(np_idf))

50000
50000


In [71]:
tf_idf_per_essay = np_train_tf_essays/np_idf

In [72]:
tf_idf_per_essay_test = np_test_tf_essays/np_idf

In [73]:
len(tf_idf_per_essay_test)

1955

In [24]:
#Implement tf-idf over all essays
tfIdfVectorizer=TfidfVectorizer(use_idf=True)
tfIdf = tfIdfVectorizer.fit_transform(np_all_essays)

In [66]:
#split tf-idf vectors and associated vocab scores into a training and testing 
#data set
embedded_training = []
embedded_testing = []
score_training = []
score_testing = []

for index, essay in enumerate(np_all_essays):
  tf_idf_per_word = tfIdf[index].toarray()[0]
  essay_score = np_all_scores[index]
  if index % 2 == 0:
    embedded_training.append(tf_idf_per_word)
    score_training.append(essay_score)
  else:
    embedded_testing.append(tf_idf_per_word)
    score_testing.append(essay_score)

np_embedded_training = np.array(embedded_training)
np_embedded_testing = np.array(embedded_testing)
np_score_training= np.array(score_training)
np_score_testing = np.array(score_testing)

#currently unused code

#df = pd.DataFrame(tfIdf[index].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
#tf_idf_per_word = np.array([data[1] for data in df.iterrows()])
#indices=np.array(tfIdfVectorizer.get_feature_names())
#df = df.sort_values('TF-IDF', ascending=False

In [56]:
#create word to index and index to word mapping to make looking up values easy
indices=np.array(tfIdfVectorizer.get_feature_names_out())
ind_to_word = {} 
word_to_ind = {}
for index, word in enumerate(indices):
  ind_to_word[index] = word
  word_to_ind[word] = index

In [ ]:
'''df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print (df.head(50))'''

'df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])\ndf = df.sort_values(\'TF-IDF\', ascending=False)\nprint (df.head(50))'

**IMPORTANT**:
To reverse engineer TF-IDF, SKlearn does a few things different.

- TF is calculated by # of times word appears in document.
- IDF is calculated by log base e ([1 + # documents] / [1 + # documents word appears in])
- our IDF = log base e ([1 + count for 'the'] / [count for each word])

Once all the words have TF-IDF scores, they are normalized by dividing by the square of all the other vectors
'''

In [57]:
#Will be useful later
idf_vector_for_every_word = tfIdfVectorizer.idf_

**READ ME for Charlie's work:**

*Main takeaway:* Building a model for only TF-IDF encodings is straightforward, but we might want to consider other options.

*   TF-IDF encodings are fixed-length (21,363 words) single-dimensional vectors.
*   No sequential (word-order) information is stored in them, so neither an RNN (including LSTM) nor CNN make sense for this input type.
*   Therefore, it makes sense to just use a simple NN (achieved 0.117 MSE on training set and 0.269 on testing).

*This approach might be good enough, and we can try to apply it or implement an RNN (LSTM) for one or two of the other metrics.*

If we want to use the RNN approach for vocab, it gets a little complicated. One option is to replace the words in each essay with each TF-IDF value, then run an LSTM on that. This removes any indication of *which* words are being used, but it will give us info on how they are used (i.e. Are lots of basic words being used together, or are rarer words being mixed in?). My conclusion is that it might not be worth this effort because we don't know if it will improve the accuracy substantially.

**Extra note: I experimented with changing the scoring method gives very promising results. Using a softmax output layer to predict the score as 0, 0.5, 1, 1.5,..., 4.5, 5 instead of relu gives better accuarcy and MSE during testing, but worse for training. Obviously an overfitting issue in this case, and I think overfitting will be the biggest challenge for everything.**

In [74]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv1D, MaxPooling1D

#number of distinct words, and therefore our input layer
num_vars = len(indices)

#at this point, we have a training set of TF-IDF vectors and a testing set, 
# as well as the associated scores. next step is to create a NN to connect them
#  and create weights

#Conv1D(input_shape=(21363), filters=20, kernel_size=5, strides=1, padding='valid', activation='relu'),
#    MaxPooling1D(pool_size=2, strides=2),
#    Conv1D(filters=40, kernel_size=5, strides=1, padding='valid', activation='relu'),
#    MaxPooling1D(pool_size=2, strides=2),

model = Sequential([
    Dense(2000, activation='relu'),
    Dropout(0.5),
    Dense(1000, activation='relu'),
    Dropout(0.5),
    Dense(500, activation='relu'),
    Dropout(0.5),
    Dense(100, activation='relu'),
    Dense(1, activation='relu')
])

model.compile(optimizer='adam',
                loss='mse',
                metrics=['accuracy'])

In [76]:
model.fit(np_embedded_training, np_score_training, batch_size=16, epochs=20)

Epoch 1/20
123/123 [==============================] - 36s 291ms/step - loss: 0.0560 - accuracy: 0.0010
Epoch 2/20
123/123 [==============================] - 27s 223ms/step - loss: 0.0537 - accuracy: 0.0010
Epoch 3/20
123/123 [==============================] - 27s 223ms/step - loss: 0.0494 - accuracy: 0.0010
Epoch 4/20
123/123 [==============================] - 28s 225ms/step - loss: 0.0462 - accuracy: 0.0010
Epoch 5/20
123/123 [==============================] - 29s 235ms/step - loss: 0.0468 - accuracy: 0.0010
Epoch 6/20
123/123 [==============================] - 27s 221ms/step - loss: 0.0450 - accuracy: 0.0010
Epoch 7/20
123/123 [==============================] - 27s 222ms/step - loss: 0.0406 - accuracy: 0.0010
Epoch 8/20
123/123 [==============================] - 27s 222ms/step - loss: 0.0363 - accuracy: 5.1125e-04
Epoch 9/20
123/123 [==============================] - 27s 222ms/step - loss: 0.0327 - accuracy: 0.0010
Epoch 10/20
123/123 [==============================] - 27s 222ms/step

In [77]:
model.evaluate(np_embedded_testing, np_score_testing)

62/62 [==============================] - 6s 89ms/step - loss: 0.2749 - accuracy: 0.0000e+00


[0.27491435408592224, 0.0]

In [28]:
import tensorflow

In [29]:
#test new model
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv1D, MaxPooling1D


In [30]:
#number of distinct words, and therefore our input layer
num_vars = len(vocab_word_to_index)

#at this point, we have a training set of TF-IDF vectors and a testing set, 
# as well as the associated scores. next step is to create a NN to connect them
#  and create weights

#Conv1D(input_shape=(21363), filters=20, kernel_size=5, strides=1, padding='valid', activation='relu'),
#    MaxPooling1D(pool_size=2, strides=2),
#    Conv1D(filters=40, kernel_size=5, strides=1, padding='valid', activation='relu'),
#    MaxPooling1D(pool_size=2, strides=2),

model2 = Sequential([
    Dense(5000, activation='relu'),
    Dropout(0.5),
    Dense(1000, activation='relu'),
    Dropout(0.5),
    Dense(500, activation='relu'),
    Dropout(0.5),
    Dense(100, activation='relu'),
    Dense(1, activation='relu')
])

model2.compile(optimizer='adam',
                loss='mse',
                metrics=['accuracy'])

In [32]:
print(len(tf_idf_per_essay))
len(np_scores_train)

1956


1956

In [33]:
model2.fit(tf_idf_per_essay, np_scores_train, batch_size=16, epochs=20)

Epoch 1/20
123/123 [==============================] - 66s 518ms/step - loss: 3.1639 - accuracy: 0.0032
Epoch 2/20
123/123 [==============================] - 63s 509ms/step - loss: 1.5439 - accuracy: 0.0032
Epoch 3/20
123/123 [==============================] - 61s 492ms/step - loss: 1.2034 - accuracy: 0.0032
Epoch 4/20
123/123 [==============================] - 58s 468ms/step - loss: 0.7640 - accuracy: 0.0032
Epoch 5/20
123/123 [==============================] - 60s 488ms/step - loss: 0.5917 - accuracy: 0.0032
Epoch 6/20
123/123 [==============================] - 58s 470ms/step - loss: 0.4306 - accuracy: 0.0032
Epoch 7/20
123/123 [==============================] - 57s 463ms/step - loss: 0.3928 - accuracy: 0.0032
Epoch 8/20
123/123 [==============================] - 57s 462ms/step - loss: 0.3810 - accuracy: 0.0032
Epoch 9/20
123/123 [==============================] - 57s 462ms/step - loss: 0.3565 - accuracy: 0.0032
Epoch 10/20
123/123 [==============================] - 58s 468ms/step - l

In [39]:
model2.evaluate(tf_idf_per_essay_test, np_scores_test)

62/62 [==============================] - 6s 93ms/step - loss: 0.3255 - accuracy: 0.0015


[0.32551485300064087, 0.0015345268184319139]

In [40]:
model2.save("common_words_tf-idf_model.h5")

- use LSTMS on word embeddings


In [74]:
#number of distinct words, and therefore our input layer
num_vars = len(vocab_word_to_index)

#at this point, we have a training set of TF-IDF vectors and a testing set, 
# as well as the associated scores. next step is to create a NN to connect them
#  and create weights

#Conv1D(input_shape=(21363), filters=20, kernel_size=5, strides=1, padding='valid', activation='relu'),
#    MaxPooling1D(pool_size=2, strides=2),
#    Conv1D(filters=40, kernel_size=5, strides=1, padding='valid', activation='relu'),
#    MaxPooling1D(pool_size=2, strides=2),

model3 = Sequential([
    Dense(2000, activation='relu'),
    Dropout(0.5),
    Dense(1000, activation='relu'),
    Dropout(0.5),
    Dense(500, activation='relu'),
    Dropout(0.5),
    Dense(100, activation='relu'),
    Dense(1, activation='relu')
])

model3.compile(optimizer='adam',
                loss='mse',
                metrics=['accuracy'])

In [65]:
tf_idf_per_essay

array([[ 24.        ,   3.54098918,  48.55919941, ...,   0.        ,
          0.        ,   0.        ],
       [ 32.        ,  14.16395673,  10.40554273, ...,   0.        ,
          0.        ,   0.        ],
       [ 24.        ,  31.86890264,   6.93702849, ...,   0.        ,
          0.        ,   0.        ],
       ...,
       [ 12.        ,   0.        ,  24.2795997 , ...,   0.        ,
          0.        ,   0.        ],
       [ 16.        ,  10.62296755,  13.87405697, ...,   0.        ,
          0.        ,   0.        ],
       [108.        ,  10.62296755,  97.11839881, ...,   0.        ,
          0.        ,   0.        ]])

In [75]:
model3.fit(tf_idf_per_essay, np_scores_train, batch_size=100, epochs=50)

Epoch 1/50
20/20 [==============================] - 6s 255ms/step - loss: 3.0916 - accuracy: 0.0028
Epoch 2/50
20/20 [==============================] - 5s 250ms/step - loss: 1.9745 - accuracy: 0.0032
Epoch 3/50
20/20 [==============================] - 5s 252ms/step - loss: 1.4483 - accuracy: 0.0032
Epoch 4/50
20/20 [==============================] - 5s 256ms/step - loss: 1.0356 - accuracy: 0.0032
Epoch 5/50
20/20 [==============================] - 5s 258ms/step - loss: 0.8050 - accuracy: 0.0032
Epoch 6/50
20/20 [==============================] - 5s 262ms/step - loss: 0.6391 - accuracy: 0.0032
Epoch 7/50
20/20 [==============================] - 5s 265ms/step - loss: 0.5174 - accuracy: 0.0032
Epoch 8/50
20/20 [==============================] - 5s 267ms/step - loss: 0.4410 - accuracy: 0.0032
Epoch 9/50
20/20 [==============================] - 5s 269ms/step - loss: 0.3999 - accuracy: 0.0032
Epoch 10/50
20/20 [==============================] - 5s 271ms/step - loss: 0.3472 - accuracy: 0.0032

In [67]:
model3.evaluate(tf_idf_per_essay_test, np_scores_test)

62/62 [==============================] - 3s 49ms/step - loss: 0.4011 - accuracy: 0.0015


[0.4010857343673706, 0.0015345268184319139]

In [76]:
model3.save("common_words_tf-idf_new_model.h5")